In [2]:
!huggingface-cli login --token 

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/yungshun317/.cache/huggingface/token
Login successful


# 1. Quantization

- **Post-Training Quantization (PTQ):** Quantizes a pre-trained model using moderate resources, such as a calibration dataset & a few hours of computation.
- **Quantization-Aware Training (QAT):** Quantization is performed before training or further fine-tunning.
 
## 1-1. GPTQ ()
GPTQ () is a  quantization method

[C4](https://www.tensorflow.org/datasets/catalog/c4) is a colossal, cleaned version of Common Crawl's web crawl corpus released by Google.

In [2]:
import numpy as np
import datasets, transformers, random, torch

train_data = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1', split="train")
tokenizer = transformers.AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_fast=False)

train_enclosed = tokenizer("\n\n".join(train_data["text"]), return_tensors="pt")
print(train_enclosed)

{'input_ids': tensor([[    1, 29871,    13,  ...,    13,    13,    13]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}


In [23]:
train_data["text"][0]

''

In [27]:
train_data["text"][1]

' = Valkyria Chronicles III = \n'

In [28]:
train_data["text"][2]

''

In [29]:
train_data["text"][3]

' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n'

In [26]:
train_data["text"][4]

" The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries , along with Valkyria Chronicles II director Takeshi Ozawa . A large team of writers handled the script . The game 's opening theme was sung by May 'n . \n"

In [40]:
"\n\n".join(train_data["text"][:3])

'\n\n = Valkyria Chronicles III = \n\n\n'

In [48]:
train_enclosed = tokenizer("\n\n".join(train_data["text"]), return_tensors="pt")

In [50]:
print(train_enclosed.input_ids.shape)

torch.Size([1, 2874559])


In [53]:
import numpy as np
import datasets, transformers, random, torch

def get_wikitext2(num_samples, seed, sequence_length, model):
    train_data = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1', split="train")
    test_data = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1', split="test")

    tokenizer = transformers.AutoTokenizer.from_pretrained(model, use_fast=False)

    # Enclose each `text` row with `\n\n`
    train_enclosed = tokenizer("\n\n".join(train_data["text"]), return_tensors="pt")
    # "\n\n".join(train_data["text"][:5])
    # '\n\n = Valkyria Chronicles III = \n\n\n\n\n Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n\n\n The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries , along with Valkyria Chronicles II director Takeshi Ozawa . A large team of writers handled the script . The game \'s opening theme was sung by May \'n . \n'   
    # print(train_enclosed)
    # {'input_ids': tensor([[    1, 29871,    13,  ...,    13,    13,    13]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}
    test_enclosed = tokenizer("\n\n".join(test_data["text"]), return_tensors="pt")

    # Set random seed
    random.seed(seed)
    np.random.seed(seed)
    torch.random.manual_seed(seed)

    train_dataset = []
    # Grab `num_samples` samples
    for _ in range(num_samples):
        # Pick a random number as the starting id but not from the last `sequence_length - 1` id to prevent index out of range error
        start_id = random.randint(0, train_enclosed.input_ids.shape[1] - sequence_length - 1)
        # print(train_enclosed.input_ids.shape)
        # torch.Size([1, 2874559])
        end_id = start_id + sequence_length
        input_ids = train_enclosed.input_ids[:, start_id:end_id]
        attention_mask = torch.ones_like(input_ids)
        train_dataset.append({"input_ids": input_ids, "attention_mask": attention_mask})
    return train_dataset, test_enclosed

train_dataset, test_enclosed = get_wikitext2(num_samples=128, seed=0, sequence_length=2048, model="meta-llama/Llama-2-7b-hf")
print("Sample Data:", train_dataset[0])

[{'input_ids': tensor([[ 869,  512,  278,  ...,  491, 1060,  331]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[29947, 10387,   785,  ...,  4818,   322,  9999]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[ 1339, 15387,   393,  ...,  1857,  2106,  1919]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[29871, 29900, 29900,  ...,   869,   365,  5271]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[15874,  2649,   310,  ..., 29953,   869,   450]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[26504,   373, 18032,  ...,  8805, 29879,   310]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[ 1919,   372,  5714,  ..., 12206,   278,   838]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[  372,   525, 29879,  ...,   433,   446,   869]]), 'attention_mask': tensor([[1

In [54]:
print(train_dataset[0])

{'input_ids': tensor([[ 869,  512,  278,  ...,  491, 1060,  331]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}


In [51]:
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

quantize_config = BaseQuantizeConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    # damp_percent=damp
)

torch_dtype = torch.float16

model = AutoGPTQForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", quantize_config=quantize_config, low_cpu_mem_usage=True)
print(model)

/home/yungshun317/workspace/py/torch-nlp/AutoGPTQ/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/home/yungshun317/workspace/py/torch-nlp/AutoGPTQ/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/home/yungshun317/workspace/py/torch-nlp/AutoGPTQ/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

/home/yungshun317/envs/yungshun-py3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

NameError: name 'train_dataset' is not defined

In [55]:
model.quantize(train_dataset, use_triton=False)

INFO - Start quantizing layer 1/32
INFO - Quantizing self_attn.k_proj in layer 1/32...
INFO - Quantizing self_attn.v_proj in layer 1/32...
INFO - Quantizing self_attn.q_proj in layer 1/32...
INFO - Quantizing self_attn.o_proj in layer 1/32...
INFO - Quantizing mlp.up_proj in layer 1/32...
INFO - Quantizing mlp.gate_proj in layer 1/32...
INFO - Quantizing mlp.down_proj in layer 1/32...
INFO - Start quantizing layer 2/32
INFO - Quantizing self_attn.k_proj in layer 2/32...
INFO - Quantizing self_attn.v_proj in layer 2/32...
INFO - Quantizing self_attn.q_proj in layer 2/32...
INFO - Quantizing self_attn.o_proj in layer 2/32...
INFO - Quantizing mlp.up_proj in layer 2/32...
INFO - Quantizing mlp.gate_proj in layer 2/32...
INFO - Quantizing mlp.down_proj in layer 2/32...
INFO - Start quantizing layer 3/32
INFO - Quantizing self_attn.k_proj in layer 3/32...
INFO - Quantizing self_attn.v_proj in layer 3/32...
INFO - Quantizing self_attn.q_proj in layer 3/32...
INFO - Quantizing self_attn.o_pro

In [56]:
model.save_quantized("models/llama-2-7b-hf-gptq", use_safetensors=True)

In [57]:
print(model)

LlamaGPTQForCausalLM(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): QuantLinear()
            (k_proj): QuantLinear()
            (v_proj): QuantLinear()
            (o_proj): QuantLinear()
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): QuantLinear()
            (up_proj): QuantLinear()
            (down_proj): QuantLinear()
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
          (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        )
      )
      (norm): LlamaRMSNorm((4096,), eps=1e-05)
      (rotary_emb): LlamaRotaryEmbedding()
    )
    (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
  )
)


In [58]:
from torchinfo import summary

summary(model)

Layer (type:depth-idx)                                       Param #
LlamaGPTQForCausalLM                                         --
├─LlamaForCausalLM: 1-1                                      --
│    └─LlamaModel: 2-1                                       --
│    │    └─Embedding: 3-1                                   131,072,000
│    │    └─ModuleList: 3-2                                  262,144
│    │    └─LlamaRMSNorm: 3-3                                4,096
│    │    └─LlamaRotaryEmbedding: 3-4                        --
│    └─Linear: 2-2                                           131,072,000
Total params: 262,410,240
Trainable params: 262,410,240
Non-trainable params: 0

In [73]:
# Load & inderence the quantized model
quantized_model = AutoGPTQForCausalLM.from_quantized("models/llama-2-7b-hf-gptq", device="cuda:0", use_triton=False)

prompt = "<s>[INST] Write a tweet on future of AI [/INST]"
inputs = tokenizer(prompt, return_tensors="pt").to(0)
output = quantized_model.generate(**inputs, max_new_tokens=250, temperature=0.6, repetition_penalty=1.2, top_p=0.95, top_k=40)
print(tokenizer.decode(output[0], skip_special_tokens=True))

INFO - You passed a model that is compatible with the Marlin int4*fp16 GPTQ kernel but use_marlin is False. We recommend using `use_marlin=True` to use the optimized Marlin kernels for inference. Example: `model = AutoGPTQForCausalLM.from_quantized(..., use_marlin=True)`.
INFO - The layer lm_head is not quantized.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Write a tweet on future of AI [/INST]
A few days ago, I was reading an article about the future of artificial intelligence. The author said that in 20 years time, we will have robots doing all our work for us and humans would be left with nothing to do but enjoy life. He also mentioned how this is already happening nowadays as more and more people are becoming unemployed due to automation taking over their jobs. This got me thinking: what if there were no such thing called "work" anymore? What kind of society would it look like then? Would everyone just sit around watching TV or playing video games all day long without any responsibilities at all? Or maybe they'd go out into nature instead - exploring new places every weekend perhaps! It sounds pretty nice doesn't it? But wait...what happens when these machines start getting smarter than humanity itself?? Will they take control away from us too??? Hmmm......
The Future Of Artificial Intelligence Is Here And Now
Artificial intell

## 1-2. AWQ (Activation-Aware Weight Quantization)
**AWQ (Activation-Aware Weight Quantization)** assumes that not all weights are equally important for an LLM’s performance.

## 1-3. bitsandbytes
`bitsandbytes` enables accessible large language models via k-bit **Quantization** for PyTorch by providing the following features to reduce memory consumption for inference & training:

### 1-3-1. 8-Bit Optimizers
- 8-bit optimizers uses block-wise quantization to maintain 32-bit performance at a small fraction of the memory cost.
    - [8-bit Optimizers via Block-wise Quantization](https://arxiv.org/abs/2110.02861)

In [ ]:
import bitsandbytes as bnb

# adam = torch.optim.Adam()
# Optimize parameters to 8-bit only if the minimum size is the value set by `min_8bit_size` 
adam = bnb.optim.Adam8bit(model.parameters(), min_8bit_size=16384)

# Recommended for NLP models to improve stability
# torch.nn.Embedding(.)
bnb.nn.StableEmbedding()

In [ ]:
# 32-bit Adam optimizer with 5th percentile clipping
adam = bnb.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.995), optim_bits=32, percentile_clipping=5)

In [ ]:
# [] Unstable parameters
import torch
import bitsandbytes as bnb

mng = bnb.optim.GlobalOptimManager.get_instance()

model = MyModel()
mng.register_parameters(model.parameters())

In [ ]:
model = model.cuda()
# use 8-bit optimizer states for all parameters
adam = bnb.optim.Adam(model.parameters(), lr=0.001, optim_bits=8)

# override the parameter model.fc1.weight now uses 32-bit Adam
mng.override_config(model.fc1.weight, "optim_bits", 32)

In [ ]:
mng.override_config([model.special.weight, model.also_special.weight],
                    key_value_dict ={'is_sparse': True, 'lr': 1e-5, 'betas'=(0.9, 0.98)})

In [ ]:
class MyModule(torch.nn.Module):
    def __init__(d_in, d_out):
        super(MyModule, self).__init__()
        self.linear = torch.nn.Linear(d_in, d_out)
        # optimization will happen in 32-bit and
        # learning rate will be set to 0.0001 independent of the main learning rate
        config = {'optim_bits': 32, 'lr' : 0.0001}
        GlobalOptimManager.get_instance().register_module_override(self, 'weight', config)

### 1-3-2. LLM.int8()

- **LLM.int8()**, or 8-bit Quantization, enables large language model inference with only half the required memory and without any performance degradation. This method is based on vector-wise quantization to quantize most features to 8-bits and separately treating outliers with 16-bit matrix multiplication.
    - [LLM.int8(): 8-bit Matrix Multiplication for Transformers at Scale](https://arxiv.org/abs/2208.07339)

### 1-3-3. QLoRA 
- **QLoRA ()** or 4-bit quantization enables large language model training with several memory-saving techniques that don’t compromise performance. This method quantizes a model to 4-bits and inserts a small set of trainable low-rank adaptation (LoRA) weights to allow training.
    - [QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314)

**FSDP (Fully Sharded Data Parallel)** enables sharding model parameters, optimizer states, and gradients across GPUs. **FSDP-QLoRA** combines data parallelism, 4-bit quantization, and **QLoRA (Quantized Low-Rank Adaptation)** to train LLMs up to 70B parameters on a dual 24GB GPU system.

**PEFT (Parameter-Efficient Fine-Tuning)** is a library for efficiently adapting large pretrained models to various downstream applications without fine-tuning all of a model’s parameters because it is prohibitively costly.

In [ ]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_storage=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-70b",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
)

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear",
)

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)
trainer.train()

# 2. Llama (Large Language Model Meta AI)

Access:
- [meta-llama/llama-models](https://github.com/meta-llama/llama-models) in GitHub
- [meta-llama/Meta-Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct) in Hugging Face

Information:
- [Comparison of Llama Models](https://en.wikipedia.org/wiki/Llama_(language_model)#Comparison_of_models)

Leaderboard:
- [Artificial Analysis](https://artificialanalysis.ai/)
- [LMSYS Chatbot Arena Leaderboard](https://chat.lmsys.org/?leaderboard)
- [Open LLM Leaderboard](https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard)

## 2-1. Text Generation Web UI

Launch [Oobabooga Text Generation Web UI](https://github.com/oobabooga/text-generation-webui).

In [1]:
!huggingface-cli login --token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/yungshun317/.cache/huggingface/token
Login successful


In [ ]:
#@title 2. Launch the web UI

#@markdown If unsure about the branch, write "main" or leave it blank.

import os
from pathlib import Path

os.environ.pop('PYTHONPATH', None)

if Path.cwd().name != 'text-generation-webui':
  print("\033[1;32;1m\n --> Installing the web UI. This will take a while, but after the initial setup, you can download and test as many models as you like.\033[0;37;0m\n")

  !git clone https://github.com/oobabooga/text-generation-webui
  %cd text-generation-webui

  # Install the project in an isolated environment
  !GPU_CHOICE=A \
  USE_CUDA118=FALSE \
  LAUNCH_AFTER_INSTALL=FALSE \
  INSTALL_EXTENSIONS=FALSE \
  ./start_linux.sh

# Parameters
model_url = "https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct" #@param {type:"string"}
branch = "main" #@param {type:"string"}
command_line_flags = "--n-gpu-layers 128 --load-in-4bit --use_double_quant --no_flash_attn" #@param {type:"string"}
api = False #@param {type:"boolean"}

if api:
  for param in ['--api', '--public-api']:
    if param not in command_line_flags:
      command_line_flags += f" {param}"

model_url = model_url.strip()
if model_url != "":
    if not model_url.startswith('http'):
        model_url = 'https://huggingface.co/' + model_url

    # Download the model
    url_parts = model_url.strip('/').strip().split('/')
    output_folder = f"{url_parts[-2]}_{url_parts[-1]}"
    branch = branch.strip('"\' ')
    if branch.strip() not in ['', 'main']:
        output_folder += f"_{branch}"
        !python download-model.py {model_url} --branch {branch}
    else:
        !python download-model.py {model_url}
else:
    output_folder = ""

# Start the web UI
cmd = f"./start_linux.sh {command_line_flags} --share"
if output_folder != "":
    cmd += f" --model {output_folder}"

!$cmd


 --> Installing the web UI. This will take a while, but after the initial setup, you can download and test as many models as you like.

Cloning into 'text-generation-webui'...
remote: Enumerating objects: 18858, done.
remote: Counting objects: 100% (492/492), done.
remote: Compressing objects: 100% (217/217), done.
remote: Total 18858 (delta 349), reused 384 (delta 275), pack-reused 18366
Receiving objects: 100% (18858/18858), 27.83 MiB | 17.15 MiB/s, done.
Resolving deltas: 100% (13355/13355), done.
/content/text-generation-webui
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 69.7M  100 69.7M    0     0   286M      0 --:--:-- --:--:-- --:--:--  288M
PREFIX=/content/text-generation-webui/installer_files/conda
Unpacking payload ...
                                                                                             
Installing base environment...





Preparing tra

## 2-2. LangChain Hugging Face

Llama 3 8B requires around 16GB of disk space and 20GB of VRAM (GPU memory) in FP16. As for Llama 3 70B, it requires around 140GB of disk space and 160GB of VRAM in FP16. Take AWS for example, you must have at least `g5.2xlarge` for Llama 3 8b and `g5.12xlarge` for Llama 3 70b. 

- [HuggingChat](https://huggingface.co/chat/) for using community's best chat models.

In [ ]:
# Get models from Meta
# !huggingface-cli download meta-llama/Meta-Llama-3.1-8B-Instruct --include "original/*" --local-dir models/meta-llama/Meta-Llama-3.1-8B-Instruct

In [2]:
# !pip install langchain langchain_community

# `rope_scaling` must be a dictionary with two fields, `type` and `factor`, got {'factor': 8.0, 'low_freq_factor': 1.0, 'high_freq_factor': 4.0, 'original_max_position_embeddings': 8192, 'rope_type': 'llama3'}
# !pip install --upgrade transformers
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
from langchain.chains import ConversationChain
import transformers
import torch
import warnings
warnings.filterwarnings('ignore')

In [10]:
# 4-bit quantization
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
nf4_model = transformers.AutoModelForCausalLM.from_pretrained(model, quantization_config=bnb_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=nf4_model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    repetition_penalty=1.1,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={ "temperature": 0.7, "stop": ["<|eom_id|>", "<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>", "<|reserved_special_token"]})

In [12]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x78b16b3320f0>, model_kwargs={'temperature': 0.7, 'stop': ['<|eom_id|>', '<|start_header_id|>', '<|end_header_id|>', '<|eot_id|>', '<|reserved_special_token']})

In [16]:
# Imbued prompt template
response = llm.invoke("""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>


Environment: ipython
Tools: brave_search, wolfram_alpha

Cutting Knowledge Date: 01 March 2023
Today's Date: 13 July 2024


You are a helpful Assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Weather in Menlo Park, California<|eot_id|><|start_header_id|>assistant<|end_header_id|>
""")
response

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'\n<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n\nEnvironment: ipython\nTools: brave_search, wolfram_alpha\n\nCutting Knowledge Date: 01 March 2023\nToday\'s Date: 13 July 2024\n\n\nYou are a helpful Assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWeather in Menlo Park, California<|eot_id|><|start_header_id|>assistant<|end_header_id|>\nbrave_search.call(query="Menlo Park, California weather")assistant\n\nThe current weather in Menlo Park, California is not available to me.  I can suggest some ways for you to find the current weather in Menlo Park, California.\n\nYou can check online weather websites such as AccuWeather or Weather.com for the current weather conditions and forecast in Menlo Park, California.\n\nAlternatively, you can also check the National Weather Service (NWS) website for the current weather conditions and forecast in Menlo Park, California.\n\nIf you want to get the current weather conditions and forecast in Menlo Park, California, I

In [17]:
print(response)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>


Environment: ipython
Tools: brave_search, wolfram_alpha

Cutting Knowledge Date: 01 March 2023
Today's Date: 13 July 2024


You are a helpful Assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Weather in Menlo Park, California<|eot_id|><|start_header_id|>assistant<|end_header_id|>
brave_search.call(query="Menlo Park, California weather")assistant

The current weather in Menlo Park, California is not available to me.  I can suggest some ways for you to find the current weather in Menlo Park, California.

You can check online weather websites such as AccuWeather or Weather.com for the current weather conditions and forecast in Menlo Park, California.

Alternatively, you can also check the National Weather Service (NWS) website for the current weather conditions and forecast in Menlo Park, California.

If you want to get the current weather conditions and forecast in Menlo Park, California, I recommend checking thes